In [1]:
import string

import aesara
import aesara.sparse
import aesara.tensor as aet
import numpy as np
import pandas as pd
import scipy.sparse as sp

from tabmat import CategoricalMatrix

import sparsedot

In [20]:
X = np.array(
    [
        [1, 2, 0, 11, 0],
        [0, 4, 5, 0, 0],
        [0, 5, 6, 7, 0],
        [0, 0, 0, 8, 0],
        [0, 0, 0, 9, 10],
    ]
)
X_sp = sp.csr_matrix(X)
offset = X_sp.indptr
column = X_sp.indices
value = X_sp.data

In [21]:
vector = np.random.normal(size=5)
result = np.zeros(5)
print(sparsedot.matrix_vector(offset, column, value, vector, result))
print(X.dot(vector))


[ -3.4005539    1.33146665   0.44659992  -1.01147474 -11.51043516]
[ -3.4005539    1.33146665   0.44659992  -1.01147474 -11.51043516]


`binarydot` implements a C function using the NumPy C-API that carries out multiplication between binary matrices and column vectors. This type of multiplication happens very often when doing statistics...

So far, this only works for the particular case where the design matrix is of the type

$$
\begin{pmatrix}
1 & 0 & \cdots & 1 & 0 \\
1 & 0 & \cdots & 0 & 1 \\
1 & 1 & \cdots & 0 & 0 \\
\vdots & \vdots & \cdots & \vdots & \vdots\\
0 & 0 & \cdots & 0 & 1
\end{pmatrix}
$$

In [3]:
import formulae

rng = np.random.default_rng(1234)

x = rng.choice(["A", "B", "C"], size=10)
y = rng.choice(["X", "Y", "Z"], size=10)
z = rng.choice(["M", "N", "O"], size=10)
df = pd.DataFrame({"x": x, "y": y, "z": z})

X = formulae.design_matrices("0 + x + y + z", df).common.design_matrix

X_indices = np.ascontiguousarray(np.vstack(np.where(X != 0)).T).astype(np.int32)
vector = rng.normal(size=X.shape[1])
result = np.zeros(X.shape[0], dtype=float)

sparsedot.binary_matrix_vector(X_indices, vector, result)

array([-1.20427091, -1.63900486, -0.42584969, -0.74500956,  1.25061673,
       -3.16130481,  2.77291668,  1.25061673,  2.77291668, -0.48267259])

In [4]:
# The "categorical variables"
strings = list(string.ascii_lowercase) + list(string.ascii_uppercase)
strings += [s * 2 for s in strings]
len(strings)

104

In [5]:
class ZeroOneMatrix:
    """
    length: The number of non-zero elements
    X_indices: The positions of the non-zero elements
    """
    def __init__(self, X):
        # The argument X is a dense array. 
        # We could use a sparse repr, but I don't know if it implies a gain in our use case.
        self.length = X.shape[0]
        self.X_indices = np.ascontiguousarray(np.vstack(np.where(X != 0)).T).astype(np.int32)
        
    def dot(self, other):
        result = np.zeros(self.length, dtype=other.dtype)
        sparsedot.binary_matrix_vector(self.X_indices, other, result)
        return result

In [17]:
x = np.random.choice(strings, size=1000)

matrix_dense = np.asarray(pd.get_dummies(x))
matrix_sparse = sp.csr_matrix(matrix_dense)
zero_one_matrix = ZeroOneMatrix(matrix_dense)
sp_matrix = sp.csr_matrix(matrix_dense)
tbmat = CategoricalMatrix(x)
y = np.arange(len(strings), dtype=np.float64)

Defining Aesara functions...

In [19]:
aet_x = aet.dmatrix("x")
aet_y = aet.dvector("y")
aet_Y = aet.dmatrix("y")

x_sparse = aesara.sparse.CSR(sp_matrix.data, sp_matrix.indices, sp_matrix.indptr, sp_matrix.shape)

aet_dot = aesara.function([aet_x, aet_y], aet.dot(aet_x, aet_y))
aet_sparse_dot = aesara.function([x_sparse, aet_Y], aesara.sparse.structured_dot(x_sparse, aet_Y))

In [20]:
%timeit zero_one_matrix.dot(y)
%timeit matrix_dense.dot(y)
%timeit sp_matrix.dot(y)
%timeit tbmat.matvec(y)
%timeit aet_dot(matrix_dense, y)
%timeit aet_sparse_dot(sp_matrix, y[:, None])

2.27 µs ± 8.81 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.39 µs ± 7.48 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
77.4 µs ± 4.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
5.79 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.75 µs ± 931 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
101 µs ± 236 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
33.7 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
x = np.random.choice(strings, size=100000)
matrix_dense = np.asarray(pd.get_dummies(x))
matrix_sparse = sp.csr_matrix(matrix_dense)

zero_one_matrix = ZeroOneMatrix(matrix_dense)
sp_matrix = sp.csr_matrix(matrix_dense)
tbmat = CategoricalMatrix(x)

%timeit zero_one_matrix.dot(y)
%timeit matrix_dense.dot(y)
%timeit sp_matrix.dot(y)
%timeit tbmat.matvec(y)
%timeit aet_dot(matrix_dense, y)
%timeit aet_sparse_dot(sp_matrix, y[:, None])

99.6 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
271 µs ± 397 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
20.6 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
190 µs ± 4.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
57.4 µs ± 740 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
22.3 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
258 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
x = np.random.choice(strings, size=10000000)
matrix_dense = np.asarray(pd.get_dummies(x))
matrix_sparse = sp.csr_matrix(matrix_dense)

zero_one_matrix = ZeroOneMatrix(matrix_dense)
sp_matrix = sp.csr_matrix(matrix_dense)
tbmat = CategoricalMatrix(x)

%timeit zero_one_matrix.dot(y)
%timeit matrix_dense.dot(y)
%timeit sp_matrix.dot(y)
%timeit tbmat.matvec(y)
%timeit aet_dot(matrix_dense, y)
%timeit aet_sparse_dot(sp_matrix, y[:, None])

39.1 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
51.2 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.86 s ± 73.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
62.3 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.6 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.67 s ± 92.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
63.5 ms ± 671 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
